In [1]:
import traitlets
import ipywidgets
from IPython.display import display

# Interfejs kamery.ipynb_checkpoints/i silników - JetRacer
from jetcam.utils import bgr8_to_jpeg
from jetcam.csi_camera import CSICamera
from jetracer.nvidia_racecar import NvidiaRacecar

# Podstawowe biblioteki Python do przetwarzania obrazów
from uuid import uuid1
import os
import json
import glob
import datetime
import numpy as np
import cv2
import time

In [2]:
#tworzenie wykorzystywanych obiektów
camera = CSICamera(width=224, height=224, capture_device=0, capture_fps = 30)
car = NvidiaRacecar()
controller = ipywidgets.Controller(index=0)

#parametryzacja widżetów
widget_width = camera.width
widget_height = camera.height

image_widget = ipywidgets.Image(format='jpeg', width=widget_width, height=widget_height)

time.sleep(1)

traitlets.dlink((camera, 'value'), (image_widget, 'value'), transform=bgr8_to_jpeg)

In [4]:
# zaprogramowanie kontrolera
left_right = traitlets.dlink((controller.axes[0], 'value'), (car, 'steering'), transform=lambda x: -x)
forward = traitlets.dlink((controller.buttons[7], 'value'), (car, 'throttle'), transform=lambda x: -x)
backward = traitlets.dlink((controller.buttons[6], 'value'), (car, 'throttle'), transform=lambda x: x)

In [5]:
blocked_dir = 'ca_dataset/blocked'
free_dir = 'ca_dataset/free'

try:
    os.makedirs(free_dir)
    os.makedirs(blocked_dir)
except FileExistsError:
    print('Directories not created because they already exist')

Directories not created because they already exist


In [6]:
button_layout = ipywidgets.Layout(width='128px', height='64px')
free_button = ipywidgets.Button(description='add free', button_style='success', layout=button_layout)
blocked_button = ipywidgets.Button(description='add blocked', button_style='danger', layout=button_layout)
free_count = ipywidgets.IntText(layout=button_layout, value=len(os.listdir(free_dir)))
blocked_count = ipywidgets.IntText(layout=button_layout, value=len(os.listdir(blocked_dir)))

In [7]:
def save_snapshot(directory):
    image_path = os.path.join(directory, str(uuid1()) + '.jpg')
    with open(image_path, 'wb') as f:
        f.write(image_widget.value)

def save_free():
    global free_dir, free_count
    save_snapshot(free_dir)
    free_count.value = len(os.listdir(free_dir))
    
def save_blocked():
    global blocked_dir, blocked_count
    save_snapshot(blocked_dir)
    blocked_count.value = len(os.listdir(blocked_dir))
    
def save_free_gamepad(change):
     if change['new']:
        global free_dir, free_count
        save_snapshot(free_dir)
        free_count.value = len(os.listdir(free_dir))
    
def save_blocked_gamepad(change):
     if change['new']:
        global blocked_dir, blocked_count
        save_snapshot(blocked_dir)
        blocked_count.value = len(os.listdir(blocked_dir))
    
controller.buttons[0].observe(save_free_gamepad, names='value')
controller.buttons[1].observe(save_blocked_gamepad, names='value')

free_button.on_click(lambda x: save_free())
blocked_button.on_click(lambda x: save_blocked())

In [8]:
#wyświetlanie widżetów
camera.running = True

display(image_widget)
display(ipywidgets.HBox([free_count, free_button]))
display(ipywidgets.HBox([blocked_count, blocked_button]))
display(controller)

Image(value=b'\xff\xd8\xff\xe0\x00\x10JFIF\x00\x01\x01\x00\x00\x01\x00\x01\x00\x00\xff\xdb\x00C\x00\x02\x01\x0…

Controller(axes=(Axis(value=0.09742879867553711), Axis(value=0.038498520851135254), Axis(value=-0.091782987117…

In [ ]:
camera.running = False